## Assignment-07 First Step of using machine learning and models.

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [12]:
import pandas as pd
filename = '../../datasource/sqlResult_1558435.csv'
content = pd.read_csv(filename, encoding='gb18030')

In [13]:
content.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [35]:
def transform(line):
    if line['source'] == '新华社':
        flag = 1
    else:
        flag = 0
    return pd.Series([line['content'], int(flag)], index=('content', 'y'))

In [28]:
df = pd.DataFrame(content)

In [41]:
data = df.apply(transform, axis=1)
len(data)

89611

In [37]:
data.head(5) # 这里为何y打印出来是浮点型

,content,y
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0.0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0.0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0.0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1.0
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0.0


### Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

In [52]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [43]:
data = data.dropna()  # 去掉记录中包含NAN的整条数据

In [47]:
corpus_content = data.content
corpus_y = data.y.values.astype(np.int)
len(corpus_content), len(corpus_y)

(87054, 87054)

In [59]:
corpus_content[:3]

0    此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1    骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2    此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
Name: content, dtype: object

In [71]:
corpus_cut = []
for i, content in tqdm(enumerate(corpus_content)):
    corpus_cut.append(' '.join(jieba.cut(content)))
len(corpus_cut)

87054it [02:36, 554.83it/s]


87054

In [73]:
vectorizer = TfidfVectorizer(max_features=300)
X = vectorizer.fit_transform(corpus_cut)

In [74]:
X.shape

(87054, 300)

In [78]:
X.toarray()
X.shape

(87054, 300)

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [87]:
# 使用 train_test_split 分割两次，将数据集分割成 train_data, validation_data, test_data
X_train, X_test, y_train, y_test = train_test_split(X, corpus_y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [89]:
X_train.shape, X_valid.shape, X_test.shape

((55714, 300), (13929, 300), (17411, 300))

In [91]:
neigh = KNeighborsClassifier(n_neighbors=5,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None)

neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [93]:
neigh.score(X_valid, y_valid)

0.9241151554311149

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

In [96]:
X_test.shape, y_test.shape

((17411, 300), (17411,))

In [97]:
y_predict = neigh.predict(X_valid)

In [98]:
p = precision_score(y_valid, y_predict)
r = recall_score(y_valid, y_predict)
f1 = f1_score(y_valid, y_predict)
print('p:{}, r:{}, f1:{}'.format(p, r, f1))

p:0.9444916892153074, r:0.9730009557183816, f1:0.9585343846847908


precision 准确率
recall    召回率
f1_score  precision and recall 的调和平均数

### Step6: 使用 test_data 对其进行新数据的判断, 你有没

In [99]:
y_predict_test = neigh.predict(X_test)

In [100]:
p_test = precision_score(y_test, y_predict_test)
r_test = recall_score(y_test, y_predict_test)
f1_test = f1_score(y_test, y_predict_test)
print('p:{}, r:{}, f1:{}'.format(p_test, r_test, f1_test))

p:0.9433288160650487, r:0.9718238355121208, f1:0.9573643410852714


### Step7: 调整不同的参数，观察变化

In [120]:
# 修改 n_neighbors 为 4，leaf_size 为 20
neigh = KNeighborsClassifier(n_neighbors=4,
    weights='uniform',
    algorithm='auto',
    leaf_size=20,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None)

neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=20, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=4, p=2,
           weights='uniform')

In [121]:
neigh.score(X_valid, y_valid)

0.9157872065474908

In [122]:
y_predict = neigh.predict(X_valid)
p = precision_score(y_valid, y_predict)
r = recall_score(y_valid, y_predict)
f1 = f1_score(y_valid, y_predict)
print('p:{}, r:{}, f1:{}'.format(p, r, f1))

p:0.9574793023068885, r:0.9487097801847723, f1:0.9530743689242709


### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

In [123]:
# 修改 weights 为 ‘distance’
neigh = KNeighborsClassifier(n_neighbors=5,
    weights='distance',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None)

neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')

In [124]:
neigh.score(X_valid, y_valid)

0.9307918730705722

In [125]:
y_predict = neigh.predict(X_valid)
p = precision_score(y_valid, y_predict)
r = recall_score(y_valid, y_predict)
f1 = f1_score(y_valid, y_predict)
print('p:{}, r:{}, f1:{}'.format(p, r, f1))

p:0.948051948051948, r:0.9767441860465116, f1:0.9621842146555782


### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [101]:
y_flag = neigh.predict(X)  # 预测所有的数据

In [102]:
y_flag.shape

(87054,)

In [111]:
content = pd.read_csv(filename, encoding='gb18030')
content_dropna = content.dropna(subset=['content'])

In [113]:
len(content_dropna), len(corpus_y)

(87054, 87054)

In [114]:
content_dropna['y_origin'] = corpus_y
content_dropna['y_predict'] = y_flag

/Users/zhangqingxuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/zhangqingxuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [118]:
copy_news = content_dropna[(content_dropna['y_origin'] == 0) & (content_dropna['y_predict'] == 1)]
len(copy_news)

3721

In [119]:
copy_news[:3]   # copy_news 即为预测出的抄袭候选者

,id,author,source,content,feature,title,url,y_origin,y_predict
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0,1
6,89611,NaN,威锋网@http://www.feng.com/,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""21""...",配骁龙660 全面屏鼻祖夏普新机酝酿中,http://www.cnbeta.com/articles/tech/623603.htm,0,1
8,89609,李杭_BJS4645,荆楚网-楚天都市报,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1600""...",武汉千余警察出动 抓获808名俊男靓女全是诈骗犯,http://news.163.com/17/0614/14/CMT9N8G80001899...,0,1


### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

数据思维是通过数据找出事务间的关系，挖掘数据中隐藏的信息，从而达到相应目的的一种思维方式。
机器学习思维是通过算法模型来提取海量数据中的特征的一种思维方式，不是简单的从输入计算输出，而是通过数据获取规律，从而达到目的。

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。